In [1]:
import pybedtools as pbt
import pandas as pd
from glob import glob
import pyranges as pr
import os
import sys
sys.path.append(os.path.abspath('../../../src/'))
import GtfBedLibrary as gbl

In [2]:
def readBed(f):
    return pd.read_csv(f, sep='\t', header=None)

def getFeatureCenters(df):
    """
    Gets centers of input features, returns bed6.
    """
    df['length'] = df[2] - df[1]
    df['middle'] = df['length'].apply(lambda x: int(x / 2) if x % 2 == 0 else int(x / 2) + 1)
    # -1 is necessary because start pos is already included
    df.loc[df[5]=='+', 'adj_start'] = df.loc[df[5]=='+', 1] + df.loc[df[5]=='+', 'middle'] - 1
    df.loc[df[5]=='-', 'adj_start'] = df.loc[df[5]=='-', 2] - df.loc[df[5]=='-', 'middle']
    df['adj_end'] = df['adj_start'] + 1
    df[['adj_start', 'adj_end']] = df[['adj_start', 'adj_end']].astype(int)
    bt = pbt.BedTool().from_dataframe(df[[0, 'adj_start', 'adj_end', 3, 4, 5]]).sort()
    return bt

def getRegions(bt, fai, window=20):
    df = bt.slop(b=window, g=fai).sort().to_dataframe(disable_auto_names=True, header=None)
    df['length'] = df[2] - df[1]
    # Remove clipped regions that extend beyond chromosome limits
    df = df.loc[df.length == df.length.max()]
    return df.iloc[:, :-1]

In [3]:
# Files

# Params
# Region around peaks
w = 100

# PABP peaks
pabpPeaks = glob('../../../data/Pabpc1Pabpc4Iclip_2022/Clippy/*Peaks*')

# LIN28A peaks
linPeaks = glob('../../../data/LIN28_220626_results/ClippyPeaks/*Peaks*')

# full 3'UTRs
utrs = pbt.BedTool('../../../data/3UtrAtlas/ThreePrimeUtrsOfMostExpressedTxInS200WT2iL.bed').sort()

# fasta index
fai = '../../../data/genomes/Goodwright_m39/GRCm39.primary_assembly.genome.fa.fai'

# Annotation
gtf = pr.read_gtf('../../../data/genomes/Goodwright_m39/gencode.vM28.primary_assembly.annotation.gtf.gz', as_df=True)
gtf = gtf.loc[gtf.Feature == 'gene', ['Chromosome', 'Start', 'End', 'gene_id', 'Score', 'Strand']]
gtf.gene_id = gtf.gene_id.apply(lambda x: x.split('.')[0])

# Expression
DfTpm = pd.read_csv('../../../data/MihaDeseq/GeneLevel_TPM_counts.csv')

# Save to
out_regions = 'ProfileRegions'
os.makedirs(out_regions, exist_ok=True)

In [4]:
# Merge LIN28A peaks and PABP peaks
linBts = [pbt.BedTool(f).sort() for f in linPeaks]
pabpcBts = [pbt.BedTool(f).sort() for f in pabpPeaks]
    # concat
catLin = linBts[0].cat(*linBts[1:], postmerge=False).sort()
catPabp = pabpcBts[0].cat(*pabpcBts[1:], postmerge=False).sort()
    # merge
mergedLin = catLin.merge(s=True, d=0, c=[4,5,6], o=['distinct', 'sum', 'distinct']).sort()
mergedPabp = catPabp.merge(s=True, d=0, c=[4,5,6], o=['distinct', 'sum', 'distinct']).sort()
    # intersect with UTRs
mergedLin = mergedLin.intersect(utrs, s=True, wa=True, u=True, nonamecheck=True).sort()
mergedPabp = mergedPabp.intersect(utrs, s=True, wa=True, u=True, nonamecheck=True).sort()
    # intersect
LinShared = mergedLin.intersect(mergedPabp, s=True, wa=True, u=True, nonamecheck=True).sort()
LinIndependent = mergedLin.intersect(mergedPabp, s=True, v=True,  nonamecheck=True).sort()
    # save peaks
LinIndependent.saveas('Independent_LinPeaks.bed.gz')
LinShared.saveas('Shared_LinPeaks.bed.gz')

<BedTool(Shared_LinPeaks.bed.gz)>

In [5]:
# Get gene loci
bed_gtf = pbt.BedTool.from_dataframe(gtf.assign(Score=0)).sort()
bed_gtf.head()

GL456210.1	9123	58882	ENSMUSG00000079800	0	-
 GL456210.1	108389	110303	ENSMUSG00000095092	0	-
 GL456210.1	123791	124928	ENSMUSG00000079192	0	+
 GL456210.1	135394	136519	ENSMUSG00000079794	0	-
 GL456210.1	147791	149707	ENSMUSG00000094799	0	+
 GL456211.1	30482	32396	ENSMUSG00000095250	0	-
 GL456211.1	54468	54840	ENSMUSG00000095787	0	-
 GL456211.1	66954	67326	ENSMUSG00000096100	0	+
 GL456211.1	113867	114758	ENSMUSG00000094054	0	+
 GL456211.1	160464	162404	ENSMUSG00000095672	0	-
 

In [6]:
# Get mean TPMs in each condition for gene-level TPMs
DfTpm.set_index('stable_gene_id', drop=True, inplace=True)
# To avoid 0-division error assign a +1 to all values
DfTpm = DfTpm + 1
DfTpm.head()
conditions = []
colnames = []
for c in DfTpm.columns:
    cond = '_'.join([c.split('_')[el] for el in [0, 2]])
    colnames.append(f'{cond}.{c}')
    conditions.append(cond)
conditions = sorted(set(conditions))
DfTpm.columns = colnames

DfTpmAveraged = pd.DataFrame()
for cond in conditions:
    DfTpmAveraged[f'{cond} Mean TPM'] = DfTpm[[c for c in DfTpm.columns if c.split('.')[0] == cond]].mean(axis='columns')
DfTpmAveraged.head()

,KO_2iL Mean TPM,KO_FCL Mean TPM,S200A_2iL Mean TPM,S200A_FCL Mean TPM,S200WT_2iL Mean TPM,S200WT_FCL Mean TPM,WT_2iL Mean TPM,WT_FCL Mean TPM
stable_gene_id,,,,,,,,
ENSMUSG00000000001,7.677950,8.112323,6.880970,6.033886,8.122433,2.797935,4.333492,3.864102
ENSMUSG00000000003,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
ENSMUSG00000000028,11.692164,9.045459,10.775650,7.133840,14.348023,4.322841,15.107630,7.196186
ENSMUSG00000000031,1.093472,1.000000,1.062915,1.569452,1.000000,1.000000,1.000000,1.178640
ENSMUSG00000000037,1.144660,1.038239,1.305144,1.137961,1.196149,1.249172,1.057581,1.086607


In [7]:
cond_of_interest = ['S200WT_FCL', 'S200A_FCL', 'S200WT_2iL']

for landmarks in ['Independent_LinPeaks.bed.gz', 'Shared_LinPeaks.bed.gz']:
    fname = landmarks.split('.')[0]
    # Read landmarks into dataframe
    df_landmarks = readBed(landmarks)
    # Drop duplicated landmarks
    df_landmarks = df_landmarks.drop_duplicates()
    # Get centers
    landmark_sites = getFeatureCenters(df_landmarks)
    # Map gene ids to features
    landmark_sites = landmark_sites.map(bed_gtf, c=4, o='distinct', s=True, nonamecheck=True)
    # Get regions around landmarks of interest
    dfRegs = getRegions(landmark_sites, fai, window=w)
    for c in cond_of_interest:
        dfRegs = dfRegs.merge(DfTpmAveraged[f'{c} Mean TPM'], left_on=6, right_index=True)
        dfRegs = dfRegs[[0, 1, 2, 6, f'{c} Mean TPM', 5]].sort_values(by=[0, 1], ascending=True)
        print(dfRegs.head(2))
        dfRegs.to_csv(f'{out_regions}/{fname}_w{w}_cond-{c}.bed.gz', sep='\t', index=False, header=None, quoting=None)


      0        1        2                   6  S200WT_FCL Mean TPM  5
0  chr1  4846537  4846738  ENSMUSG00000033845            15.182613  -
1  chr1  4846573  4846774  ENSMUSG00000033845            15.182613  -
      0        1        2                   6  S200A_FCL Mean TPM  5
0  chr1  4846537  4846738  ENSMUSG00000033845            17.58874  -
1  chr1  4846573  4846774  ENSMUSG00000033845            17.58874  -
      0        1        2                   6  S200WT_2iL Mean TPM  5
0  chr1  4846537  4846738  ENSMUSG00000033845            24.556698  -
1  chr1  4846573  4846774  ENSMUSG00000033845            24.556698  -
      0        1        2                   6  S200WT_FCL Mean TPM  5
7  chr1  5232583  5232784  ENSMUSG00000033793             4.794732  +
8  chr1  5232620  5232821  ENSMUSG00000033793             4.794732  +
      0        1        2                   6  S200A_FCL Mean TPM  5
7  chr1  5232583  5232784  ENSMUSG00000033793            7.241819  +
8  chr1  5232620  5232821